In [1]:
import os
import os.path as osp

import numpy as np

import open3d as o3d

import utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd_path = "data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=10, radius=0.2)[0] # (pcd, new indexed from old)

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

In [3]:
print(points.shape)
print(colors.shape)

(13201, 3)
(0, 3)


# height filtration

In [4]:
# pivot = utils.get_average_pivot(points[:, 2])
# points = points[points[:, 2] < pivot * 1.5]

# covariance feature

In [5]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

feat_list = None
feat_list_dump_file = f"data/kinth/pickle/feat_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_list_dump_file):
#     with open(feat_list_dump_file, "rb") as f:
#         feat_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_list_con = [None] * num_worker
with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_radius,
            points[i * batch_size : min((i + 1) * batch_size, len(points))],
            2.0
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, _ = future.result()
        feat_list_con[future_dict[future]] = feat_list
feat_list = np.concatenate(feat_list_con, axis=0)
    
    # with open(feat_list_dump_file, "wb") as f:
    #     pickle.dump(feat_list, f)

eigval progress: 100%|███████████████████████████████████████| 3298/3298 [00:00<00:00, 14166.27it/s]


In [6]:
feat_rgb = (feat_list - feat_list.min(axis=1).reshape(-1,1) + 1e-3) / (feat_list.max(axis=1).reshape(-1, 1) + 1e-3)

feat_rgb = (feat_rgb * 255.0).astype(np.int32)
print(feat_rgb)

[[ 15 167   0]
 [ 15 171   0]
 [ 21 177   0]
 ...
 [108 238   0]
 [  0 247  51]
 [ 50 242   0]]


In [7]:
utils.npy2ply(points, feat_rgb, "sample.ply")